In [154]:
class Cup(object):
    def __init__(self, cup_id: int):
        self.cup_id = cup_id
        self.next = None
    def __repr__(self) -> str:
        return str(self.cup_id)

class CupsCircle(object):
    def __init__(self, data: str):
        prev_cup = None
        for i, cup_id in enumerate(data):
            cup = Cup(int(cup_id))
            if i == 0:
                self.current = cup
            if prev_cup:
                prev_cup.next = cup
            prev_cup = cup
        prev_cup.next = self.current
    
    def take_tripple(self) -> Cup:
        tripple_begin = self.current.next
        tripple_end = self.current.next.next.next
        self.current.next = tripple_end.next
        tripple_end.next = None
        return tripple_begin
    
    def find_destination(self) -> Cup:
        current_id = self.current.cup_id
        min_gap = 1 << 31
        max_id = -1
        max_id_cup = None
        destination_cup = None
        cup_ptr = self.current.next
        while cup_ptr != self.current:
            gap = current_id - cup_ptr.cup_id
            if gap == 1:
                return cup_ptr
            if gap > 0 and gap < min_gap:
                min_gap = gap
                destination_cup = cup_ptr
            if gap < 0 and cup_ptr.cup_id > max_id:
                max_id = cup_ptr.cup_id
                max_id_cup = cup_ptr
            cup_ptr = cup_ptr.next
        return destination_cup if destination_cup else max_id_cup
    
    def put_tripple_back(self, dest: Cup, tripple: Cup):
        tripple_end = tripple.next.next
        tripple_end.next = dest.next
        dest.next = tripple
    
    def one_move(self):
        tripple = self.take_tripple()
        destination = self.find_destination()
        self.put_tripple_back(destination, tripple)
        self.current = self.current.next
    
    def __repr__(self) -> str:
        cup_ptr = self.current
        result = ''
        while True:
            result += f'{cup_ptr} => '
            cup_ptr = cup_ptr.next
            if cup_ptr == self.current:
                return result
    
    def no_one_serie(self) -> str:
        cup_ptr = self.current
        while cup_ptr.cup_id != 1:
            cup_ptr = cup_ptr.next
            cup_ptr = cup_ptr.next
        cup_ptr = cup_ptr.next
        result = ''
        while cup_ptr.cup_id != 1:
            result += str(cup_ptr)
            cup_ptr = cup_ptr.next
        return result

In [155]:
def part1_solution(data: str, moves: int) -> str:
    circle = CupsCircle(data)
    for _ in range(moves):
        circle.one_move()
    return circle.no_one_serie()

In [156]:
part1_solution('389125467', 10)

'92658374'

In [157]:
part1_solution('364297581', 100)

'47382659'

In [158]:
%timeit part1_solution('364297581', 10000)

37.8 ms ± 405 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [283]:
class Cup(object):
    def __init__(self, cup_id: int):
        self.cup_id = cup_id
        self.next = None
        self.prev = None
        self.less_one = None
        self.is_chained = True
    def __repr__(self) -> str:
        return str(self.cup_id)

class CupsCircle(object):
    def __init__(self, data: str, end: int=1_000_001):
        prev_cup = None
        cups_dict = {}
        for i, cup_id in enumerate(data):
            cup = Cup(int(cup_id))
            if i == 0:
                self.current = cup
            if prev_cup:
                prev_cup.next = cup
                cup.prev = prev_cup
            if cup_id == '1':
                self.first_cup = cup
            cups_dict[int(cup_id)] = cup
            prev_cup = cup
#         print(cups_dict)
        for cup_id, cup in cups_dict.items():
            if cup_id > 1:
                cup.less_one = cups_dict[cup_id - 1]
        less_one_cup = cups_dict[len(data)]
        for cup_id in range(len(data) + 1, end):
            cup = Cup(cup_id)
            prev_cup.next = cup
            cup.prev = prev_cup
            cup.less_one = less_one_cup
            prev_cup = cup
            less_one_cup = cup
        prev_cup.next = self.current
        self.current.prev = prev_cup
        self.first_cup.less_one = less_one_cup
#         self.print_less_order()
    
    def take_tripple(self) -> Cup:
        tripple_begin = self.current.next
        cup_ptr = tripple_begin
        for _ in range(2):
            cup_ptr.is_chained = False
            cup_ptr = cup_ptr.next
        cup_ptr.is_chained = False
        self.current.next = cup_ptr.next
        cup_ptr.next.prev = self.current
        tripple_begin.prev = None
        cup_ptr.next = None
        return tripple_begin
    
    def find_destination(self) -> Cup:
        cup_ptr = self.current.less_one
        while not cup_ptr.is_chained:
            cup_ptr = cup_ptr.less_one
#         self.print_less_order()
        return cup_ptr
    
    def put_tripple_back(self, dest: Cup, tripple: Cup):
        cup_ptr = tripple
        for _ in range(2):
            cup_ptr.is_chained = True
            cup_ptr = cup_ptr.next
        cup_ptr.is_chained = True
        cup_ptr.next = dest.next
        dest.next.prev = cup_ptr
        dest.next = tripple
        tripple.prev = dest
    
    def one_move(self):
        tripple = self.take_tripple()
        destination = self.find_destination()
        self.put_tripple_back(destination, tripple)
        self.current = self.current.next
    
    def __repr__(self) -> str:
        cup_ptr = self.current
        result = ''
        while True:
            result += f'{cup_ptr} => '
            cup_ptr = cup_ptr.next
            if cup_ptr == self.current:
                return result
    
    def less_one_print(self):        
        cup_ptr = self.first_cup.less_one
        while True:
            if cup_ptr.is_chained:
                print(cup_ptr, end=' => ')
            if cup_ptr == self.first_cup:
                break
            cup_ptr = cup_ptr.less_one
        print()
    
    def rev_print(self) -> str:
        cup_ptr = self.current
        while True:
            print(cup_ptr, end=' <= ')
            cup_ptr = cup_ptr.prev
            if cup_ptr == self.current:
                break
        print()

In [291]:
def part2_solution(circle: CupsCircle, moves: int=10_000_000) -> int:
    for _ in range(moves):
        circle.one_move()
    print(circle.first_cup.prev, circle.first_cup.prev.prev)
    print(circle.first_cup.next, circle.first_cup.next.next)
    return circle.first_cup.next.cup_id * circle.first_cup.next.next.cup_id

In [292]:
circle = CupsCircle('389125467', 101)
for _ in range(100):
    circle.one_move()
# circle.rev_print()
circle.less_one_print()
circle

100 => 99 => 98 => 97 => 96 => 95 => 94 => 93 => 92 => 91 => 90 => 89 => 88 => 87 => 86 => 85 => 84 => 83 => 82 => 81 => 80 => 79 => 78 => 77 => 76 => 75 => 74 => 73 => 72 => 71 => 70 => 69 => 68 => 67 => 66 => 65 => 64 => 63 => 62 => 61 => 60 => 59 => 58 => 57 => 56 => 55 => 54 => 53 => 52 => 51 => 50 => 49 => 48 => 47 => 46 => 45 => 44 => 43 => 42 => 41 => 40 => 39 => 38 => 37 => 36 => 35 => 34 => 33 => 32 => 31 => 30 => 29 => 28 => 27 => 26 => 25 => 24 => 23 => 22 => 21 => 20 => 19 => 18 => 17 => 16 => 15 => 14 => 13 => 12 => 11 => 10 => 9 => 8 => 7 => 6 => 5 => 4 => 3 => 2 => 1 => 


56 => 60 => 61 => 64 => 67 => 71 => 72 => 75 => 50 => 63 => 54 => 59 => 53 => 58 => 62 => 69 => 76 => 66 => 70 => 73 => 74 => 83 => 87 => 88 => 92 => 93 => 96 => 99 => 13 => 16 => 17 => 31 => 22 => 27 => 82 => 95 => 86 => 91 => 85 => 90 => 94 => 8 => 3 => 5 => 98 => 9 => 12 => 7 => 4 => 14 => 6 => 11 => 1 => 10 => 19 => 23 => 28 => 29 => 32 => 20 => 26 => 30 => 37 => 41 => 42 => 34 => 25 => 33 => 36 => 45 => 48 => 49 => 47 => 46 => 52 => 57 => 65 => 68 => 77 => 80 => 81 => 79 => 78 => 84 => 89 => 97 => 100 => 15 => 2 => 24 => 35 => 18 => 21 => 44 => 43 => 38 => 40 => 51 => 55 => 39 => 

In [293]:
circle = CupsCircle('364297581', 0)
for _ in range(100):
    circle.one_move()
circle.rev_print()
circle

6 <= 2 <= 8 <= 3 <= 7 <= 4 <= 1 <= 9 <= 5 <= 


6 => 5 => 9 => 1 => 4 => 7 => 3 => 8 => 2 => 

In [294]:
circle = CupsCircle('389125467')
part2_solution(circle)

161909 907743
934001 159792


149245887792

In [295]:
circle = CupsCircle('364297581')
part2_solution(circle)

541770 58848
257760 163997


42271866720